In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import struct
import numpy as np
import os
import sys

## Adaptive sample training

In [2]:
feature_dim = 100;
categories = 200000;
nsamples = 2000;

In [43]:
with open('/private/home/qiantong/tmp/lm_logs/adsm_test/adsm_input_b.bin', 'rb') as f:
    data = f.read()
    h_input = struct.unpack('f' * feature_dim * nsamples, data)
    print(len(h_input))
    
with open('/private/home/qiantong/tmp/lm_logs/adsm_test/adsm_target_b.bin', 'rb') as f:
    data = f.read()
    h_target = struct.unpack('i' * nsamples, data)
    print(len(h_target))

input = torch.zeros((feature_dim * nsamples,), dtype=torch.float32)
input = input.new_tensor(h_input).reshape(nsamples, feature_dim)
print(input[0])
print(input[250])
print(input[350])

target = torch.zeros((nsamples,), dtype=torch.int32)
target = target.new_tensor(h_target).to(torch.int64)
print(target[:10])

200000
2000
tensor([0.8344, 0.5809, 0.4471, 0.6506, 0.8366, 0.7644, 0.7056, 0.3341, 0.7679,
        0.2162, 0.3955, 0.1304, 0.7658, 0.5979, 0.1689, 0.9357, 0.6382, 0.6445,
        0.8030, 0.0611, 0.5084, 0.8336, 0.8747, 0.5925, 0.2188, 0.2527, 0.2798,
        0.6364, 0.3410, 0.9844, 0.5321, 0.9699, 0.6615, 0.3889, 0.3945, 0.1447,
        0.2450, 0.0102, 0.1717, 0.7890, 0.1923, 0.5241, 0.0500, 0.3537, 0.8072,
        0.7991, 0.6464, 0.9184, 0.8881, 0.4741, 0.6913, 0.5796, 0.2668, 0.6899,
        0.2302, 0.6519, 0.2830, 0.2789, 0.4707, 0.8481, 0.2815, 0.8284, 0.2369,
        0.1835, 0.9035, 0.3367, 0.1337, 0.6613, 0.4437, 0.8792, 0.5249, 0.6371,
        0.5496, 0.3567, 0.2098, 0.3443, 0.6151, 0.3868, 0.9220, 0.5020, 0.4515,
        0.6307, 0.2177, 0.3458, 0.1999, 0.6734, 0.9900, 0.4931, 0.8068, 0.2970,
        0.3995, 0.0382, 0.4447, 0.9714, 0.1020, 0.1118, 0.5209, 0.5527, 0.3575,
        0.0543])
tensor([0.1119, 0.1876, 0.3776, 0.1045, 0.6737, 0.0850, 0.9660, 0.5560, 0.7579,
        0.7

In [110]:
class test_module(nn.Module):
    def __init__(self, feature_dim=100, categories=200000, cutoffs={10000, 50000}, load_init_params=True):
        super().__init__()
        self.adsm = nn.AdaptiveLogSoftmaxWithLoss(feature_dim, categories, cutoffs=cutoffs, div_value=4)
        self.lin1 = nn.Linear(feature_dim, feature_dim, False)
        self.lin2 = nn.Linear(feature_dim, feature_dim, False)
        
        if load_init_params:
            ## nn
            for i, param in enumerate(self.lin1.parameters()):
                with open('/private/home/qiantong/tmp/lm_logs/adsm_test/nn_param_0.bin'.format(i), 'rb') as f:
                    data = f.read()
                    param.data = param.data.new_tensor(struct.unpack('f' * param.numel(), data)).reshape(param.size(1), param.size(0))
                    param.data = param.data.transpose(0, 1)
                print(param.data[0])
                print(type(param.data), param.size())
                
            for i, param in enumerate(self.lin2.parameters()):
                with open('/private/home/qiantong/tmp/lm_logs/adsm_test/nn_param_1.bin'.format(i), 'rb') as f:
                    data = f.read()
                    param.data = param.data.new_tensor(struct.unpack('f' * param.numel(), data)).reshape(param.size(1), param.size(0))
                    param.data = param.data.transpose(0, 1)
                print(param.data[0])
                print(type(param.data), param.size())
                
            ## adsm
            for i, param in enumerate(self.adsm.parameters()):
                with open('/private/home/qiantong/tmp/lm_logs/adsm_test/adsm_param_{}.bin'.format(i), 'rb') as f:
                    data = f.read()
                    param.data = param.data.new_tensor(struct.unpack('f' * param.numel(), data)).reshape(param.size(1), param.size(0))
                    param.data = param.data.transpose(0, 1)
                print(param.data[0])
                print(type(param.data), param.size())
     
    def forward(self, x, y):
        res = self.lin1(x)
        res = F.relu(res)
        res = self.lin2(res)
        res = self.adsm(res, y)

        return res


In [113]:
adsm = test_module(load_init_params=False).cuda()
optimizer = optim.SGD(adsm.parameters(), lr=0.01)

print(input.shape)
print(target.shape)

input = input.cuda()
target = target.cuda()

for epoch in range(500):
    optimizer.zero_grad()
    loss = adsm(input, target)
    loss[1].backward()
    optimizer.step()
    print(loss[1].item())

torch.Size([2000, 100])
torch.Size([2000])
20.2977352142334
20.27950096130371
20.26127052307129
20.24294662475586
20.22442626953125
20.205629348754883
20.186460494995117
20.16681671142578
20.146608352661133
20.12571144104004
20.104005813598633
20.08135986328125
20.05766487121582
20.032773971557617
20.00652503967285
19.978734970092773
19.94921875
19.917741775512695
19.884084701538086
19.848007202148438
19.8092098236084
19.767379760742188
19.722148895263672
19.673126220703125
19.619869232177734
19.561861038208008
19.49851417541504
19.429166793823242
19.353044509887695
19.26930046081543
19.176939010620117
19.07480239868164
18.96152114868164
18.8355712890625
18.695125579833984
18.538036346435547
18.361814498901367
18.163557052612305
17.939800262451172
17.686458587646484
17.39871597290039
17.07103157043457
16.697227478027344
16.271121978759766
15.788512229919434
15.252924919128418
14.689804077148438
14.168183326721191
13.785895347595215
13.572054862976074
13.463436126708984
13.3992776870727

12.197430610656738
12.197376251220703
12.197324752807617
12.197272300720215
12.197218894958496
12.197166442871094
12.197113037109375
12.197061538696289
12.19700813293457
12.196955680847168
12.196903228759766
12.196849822998047
12.196797370910645
12.196745872497559
12.196693420410156
12.196640968322754
12.196588516235352
12.196537017822266
12.196483612060547
12.196431159973145
12.196379661560059
12.196327209472656
12.196274757385254
12.196223258972168
12.196171760559082
12.19611930847168
12.196066856384277
12.196015357971191
12.195963859558105
12.195910453796387
12.1958589553833
12.195807456970215
12.195754051208496
12.19570255279541
12.195651054382324
12.195600509643555
12.195547103881836
12.19549560546875
12.195444107055664
12.195393562316895
12.195341110229492
12.195289611816406
12.19523811340332
12.195185661315918
12.195135116577148
12.195083618164062
12.195032119750977
12.194979667663574
12.194928169250488
12.194877624511719
12.19482707977295
12.194774627685547
12.194723129272461
1

## Grad check

In [5]:
sys.path.insert(0, "/checkpoint/antares/experiments/librispeech_lms/fairseq-py")

from fairseq.data import Dictionary
from fairseq.models.fconv_lm import FConvLanguageModel
from fairseq.models.transformer_lm import TransformerLanguageModel

In [6]:
model_path = "/checkpoint/antares/old_checkpoint02/2018-11-13/word_lm_librispeech.fp16.fconv_lm.nag.lr0.5.lrs=fixed.clip0.1.emb128.layers_0_1024_5_0_4096_1_3.drop0.1.wd1e-07.mxtk1024.smptk1024.bm=none.crt=adaptive_loss.adap=10000_50000_200000.seed42.ngpu8/checkpoint_best.pt"

def load_lm(lm_path, model_type, dict_path):
    path, checkpoint = os.path.split(lm_path)
    if model_type == "convlm":
        model_handle = FConvLanguageModel.from_pretrained(
            path, checkpoint, os.path.split(dict_path)[0]
        )
    elif model_type == "transformer":
        model_handle = TransformerLanguageModel.from_pretrained(
            path, checkpoint, os.path.split(dict_path)[0]
        )
    else:
        raise Exception(
            "Unsupported language model type: use 'convlm' or 'transformer' models"
        )
    model = model_handle.models[0].decoder.cuda()
    model.eval()
    print(model)
    return model

def compute_loss(model, sample, target):
    reduce = 'none'
    adaptive_softmax = model.adaptive_softmax
    net_output = model.forward(sample)[0]
    
    logits, target = adaptive_softmax(net_output, target)
    loss = net_output.new(1).zero_()

    for i in range(len(target)):
        if target[i] is not None:
            assert (target[i].min() >= 0 and target[i].max() <= logits[i].size(1))
            loss += F.cross_entropy(
                logits[i],
                target[i],
                ignore_index=1,
                reduction='sum' if reduce else 'none',
            )

    return loss

In [7]:
raw_model = load_lm(model_path, "convlm", "/checkpoint/antares/released_models/lexicon_free/librispeech/word_gcnn/dict.txt")


loading archive file /checkpoint/antares/old_checkpoint02/2018-11-13/word_lm_librispeech.fp16.fconv_lm.nag.lr0.5.lrs=fixed.clip0.1.emb128.layers_0_1024_5_0_4096_1_3.drop0.1.wd1e-07.mxtk1024.smptk1024.bm=none.crt=adaptive_loss.adap=10000_50000_200000.seed42.ngpu8
loading archive file /checkpoint/antares/released_models/lexicon_free/librispeech/word_gcnn
| dictionary: 221452 types
Namespace(adaptive_softmax_cutoff='10000,50000,200000', adaptive_softmax_dropout=0, arch='fconv_lm', bucket_cap_mb=150, clip_norm=0.1, criterion='adaptive_loss', data='/checkpoint/antares/released_models/lexicon_free/librispeech/word_gcnn', ddp_backend='no_c10d', decoder_attention='False', decoder_embed_dim=128, decoder_layers='[(512, 5)] + [(128, 1, 0), (128, 5, 0), (512, 1, 3)] * 3 + [(512, 1, 0), (512, 5, 0), (1024, 1, 3)] * 3 + [(1024, 1, 0), (1024, 5, 0), (2048, 1, 3)] * 6 + [(1024, 1, 0), (1024, 5, 0), (4096, 1, 3)]', device_id=0, distributed_backend='nccl', distributed_init_host='localhost', distributed_

In [10]:
import copy

model = copy.deepcopy(raw_model)
model.adaptive_softmax = None
model.eval()

input_x = [2, 3, 4, 5, 6, 7, 8, 9, 10, 2]
x = torch.LongTensor(np.array(input_x[0:-1]).reshape(1, len(input_x) - 1)).cuda()
target = torch.LongTensor(np.array(input_x[1:]).reshape(1, len(input_x) - 1)).cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

for _ in range(2):
    optimizer.zero_grad()
#     loss = compute_loss(model, x, target)
    print(x)
    out = model.forward(x)[0]
    loss = torch.sum(out * out)
    print(loss)
    loss.backward()
    
    for p in model.embed_tokens.parameters():
        print(p.shape)
        print(p[2][:10])
        print(p.grad.shape)
        print(p.grad[2][:10])
        
    optimizer.step()


FConvDecoder(
  (embed_tokens): Embedding(221452, 128, padding_idx=1)
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (projections): ModuleList(
    (0): None
    (1): None
    (2): None
    (3): None
    (4): None
    (5): None
    (6): None
    (7): None
    (8): None
    (9): None
    (10): None
    (11): None
    (12): Linear(in_features=512, out_features=1024, bias=True)
    (13): None
    (14): None
    (15): None
    (16): None
    (17): None
    (18): None
    (19): None
    (20): None
    (21): Linear(in_features=1024, out_features=2048, bias=True)
    (22): None
    (23): None
    (24): None
    (25): None
    (26): None
    (27): None
    (28): None
    (29): None
    (30): None
    (31): None
    (32): None
    (33): None
    (34): None
    (35): None
    (36): None
    (37): None
    (38): None
    (39): Linear(in_features=2048, out_features=4096, bias=True)
  )
  (convolutions): ModuleList(
    (0): LinearizedConvolution(512, 1024, kernel_size=(5,), paddin